In [1]:
import numpy as np
import pandas as pd
import time
import os
import tensorflow as tf
tf.config.run_functions_eagerly(False)
import plotting_funcs as pf
import config
import prepare_data_for_clustering_utils as prep

#from VaDER.vader import VADER
#save_path = os.path.join('test_vader', 'vader.ckpt')
#np.random.seed(123)

In [ ]:
x,y = prep.from_results_folder_PATH_to_arrays(features=['centroids','morphologies'],ts_len=16,cut_longer_ts=True,save=True,name_ext="_All16")
x.shape

In [3]:
X = np.load('../npy_files/features_All12.npy')
y = np.load('../npy_files/labels_All12.npy')
y_letter = np.array([well[:1] for well in y])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labs = le.fit_transform(y)
labs_by_letter = le.fit_transform(y_letter)
print(len(X),len(y))
X_train, y_train = X, labs

14330 14330


In [4]:
pd.Series(y_letter).value_counts()

D    5314
G    4022
F    2827
E    2167
dtype: int64

In [5]:
pd.Series(y).value_counts()

G4     1867
F4     1697
D7     1604
D3     1477
D4     1316
D2      917
G2      896
E6      818
F3      749
E8      591
G3      498
G5      435
F2      381
E7      371
G6      326
E9      255
E10     132
dtype: int64

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

In [ ]:
W_train = None
vader = VADER(X_train=X_train, W_train=W_train, save_path=save_path, n_hidden=[60,32,4], k=3,
              learning_rate=1e-3, output_activation=None, recurrent=True, cell_type="GRU", batch_size=64)
# pre-train without latent loss
start = time.time()
vader.pre_fit(n_epoch=64, verbose=True)
# train with latent loss
vader.fit(n_epoch=64, verbose=True)
end = time.time()
print("Elapsed: ", end - start)

In [ ]:
# get the clusters
c = vader.cluster(X_train)
# get the re-constructions
p = vader.predict(X_train)
# compute the loss given the network
l = vader.get_loss(X_train)

In [ ]:
print(pd.Series(c).value_counts())

In [ ]:
latent = vader.map_to_latent(X_train)

In [ ]:
hex, hex_dict = pf.plot_clustering(latent,labs_by_letter)

In [ ]:
X_C0 = np.array([x for x,clust in zip(X_train,c) if clust == 0])
X_C1 = np.array([x for x,clust in zip(X_train,c) if clust == 1])
X_C2 = np.array([x for x,clust in zip(X_train,c) if clust == 2])

In [ ]:
n_reps = 3
representatives_C0 = X_C0[np.random.choice(X_C0.shape[0], n_reps, replace=False)]
representatives_C1 = X_C1[np.random.choice(X_C1.shape[0], n_reps, replace=False)]
representatives_C2 = X_C2[np.random.choice(X_C2.shape[0], n_reps, replace=False)]

In [ ]:
plot_representatives([representatives_C0,representatives_C1,representatives_C2])

## specific well/gene group clustering

In [ ]:
X_G4 = np.array([x for x,well in zip(X,y) if well == "G4"])
specific_c = vader.cluster(X_G4)
specific_latent = vader.map_to_latent(X_G4)
print(pd.Series(specific_c).value_counts())

In [ ]:
X_F4 = np.array([x for x,well in zip(X,y) if well == "E6"])
specific_c2 = vader.cluster(X_F4)
specific_latent2 = vader.map_to_latent(X_F4)
print(pd.Series(specific_c2).value_counts())

In [ ]:
plot_dual_clustering(specific_latent,specific_c,specific_latent2,specific_c2,hex)

## transformer 

In [ ]:
vader = VADER(X_train=X_train, W_train=W_train, save_path=save_path, n_hidden=[64,4], k=3,
              learning_rate=1e-3, output_activation=None, recurrent=True, cell_type="Transformer", batch_size=64,
              cell_params={'d_model': 4, 'num_layers': 1, 'num_heads': 1, 'dff': 16, 'rate': 0.0})
# pre-train without latent loss
start = time.time()
vader.pre_fit(n_epoch=50, verbose=True)
# train with latent loss
vader.fit(n_epoch=50, verbose=True)
end = time.time()
print("Elapsed: ", end - start)